<a href="https://colab.research.google.com/github/Joh-Ishimwe/Data-Preprocessing/blob/master/Part_2_Merging_Datasets_with_Transitive_Properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

try:
    transactions_df = pd.read_csv('customer_transactions_augmented.csv')
    profiles_df = pd.read_csv('customer_social_profiles.csv')
    mapping_df = pd.read_csv('id_mapping.csv')
    print("Datasets loaded successfully!")
except FileNotFoundError:
    print("One or more dataset files not found. Please ensure they are uploaded to Colab.")
    exit()

    print("\n--- Transactions Data ---")
print(transactions_df.head())
print(transactions_df.info())

print("\n--- Profiles Data ---")
print(profiles_df.head())
print(profiles_df.info())

print("\n--- Mapping Data ---")
print(mapping_df.head())
print(mapping_df.info())

Datasets loaded successfully!
   customer_id_legacy  transaction_id  purchase_amount  customer_rating  \
0                 100            1130       384.920989         4.218168   
1                 100            1113       175.928161         4.000000   
2                 100            1147       384.344822         4.600000   
3                 101            1059       425.420569         2.408380   
4                 101            1059       414.524957         2.412102   

   product_category_encoded purchase_date product_category  purchase_month  \
0                         0    2024-01-10            Books               1   
1                         1    2024-04-22         Clothing               4   
2                         0    2024-05-26            Books               5   
3                         0    2024-02-26        Groceries               2   
4                         0    2024-02-27         Clothing               2   

   avg_purchase_amount  days_since_last_purchase  

#Prepare the Mapping DataFrame

In [4]:
mapping_df = mapping_df.rename(columns={'customer_id_legacy': 'legacy_id'})
print("\nMapping DataFrame after renaming column:")
print(mapping_df.head())


Mapping DataFrame after renaming column:
   legacy_id customer_id_new
0        195            A105
1        161            A118
2        192            A156
3        157            A168
4        166            A102


#Merge the Transactions DataFrame with the Mapping DataFrame

In [5]:
transactions_mapped = pd.merge(transactions_df, mapping_df, left_on='customer_id_legacy', right_on='legacy_id', how='left')
print("\nTransactions DataFrame merged with mapping data:")
print(transactions_mapped.head())


Transactions DataFrame merged with mapping data:
   customer_id_legacy  transaction_id  purchase_amount  customer_rating  \
0                 100            1130       384.920989         4.218168   
1                 100            1113       175.928161         4.000000   
2                 100            1147       384.344822         4.600000   
3                 101            1059       425.420569         2.408380   
4                 101            1059       425.420569         2.408380   

   product_category_encoded purchase_date product_category  purchase_month  \
0                         0    2024-01-10            Books               1   
1                         1    2024-04-22         Clothing               4   
2                         0    2024-05-26            Books               5   
3                         0    2024-02-26        Groceries               2   
4                         0    2024-02-26        Groceries               2   

   avg_purchase_amount  days_s

In [18]:
final_merged_df_cleaned = final_merged_df.dropna(subset=['customer_id_new']).copy()
print("\nCleaned merged DataFrame (after dropping unmapped transactions):")
print(final_merged_df_cleaned.head())
print(final_merged_df_cleaned.info())


Cleaned merged DataFrame (after dropping unmapped transactions):
   customer_id_legacy  transaction_id  purchase_amount  customer_rating  \
3                 101            1059       425.420569          2.40838   
4                 101            1059       425.420569          2.40838   
5                 101            1059       425.420569          2.40838   
6                 101            1059       425.420569          2.40838   
7                 101            1059       425.420569          2.40838   

   product_category_encoded purchase_date product_category  purchase_month  \
3                         0    2024-02-26        Groceries               2   
4                         0    2024-02-26        Groceries               2   
5                         0    2024-02-26        Groceries               2   
6                         0    2024-02-26        Groceries               2   
7                         0    2024-02-26        Groceries               2   

   avg_purchas

#Handling Potential One-to-Many Mappings

In [14]:
# Check for one-to-many mappings in id_mapping.csv
legacy_counts = mapping_df['legacy_id'].value_counts()
conflicts = legacy_counts[legacy_counts > 1]
print(f"\nNumber of legacy IDs with multiple new IDs: {len(conflicts)}")

if not conflicts.empty:
    print("Legacy IDs with multiple mappings:")
    print(conflicts)
    print("\nHandling conflicts by keeping the first mapping encountered for each legacy ID.")
    # Keep only the first occurrence of each legacy_id in the mapping
    mapping_df_unique = mapping_df.drop_duplicates(subset=['legacy_id'], keep='first')

    # Remerge using the unique mapping
    transactions_mapped_unique = pd.merge(transactions_df, mapping_df_unique, left_on='customer_id_legacy', right_on='legacy_id', how='left')
    final_merged_df = pd.merge(transactions_mapped_unique, profiles_df, on='customer_id_new', how='left')

    print("Final merged DataFrame after handling one-to-many mappings:")
    print(final_merged_df.head())
    print(final_merged_df.info())
else:
    print("No one-to-many mappings found in the id_mapping.csv.")


Number of legacy IDs with multiple new IDs: 49
Legacy IDs with multiple mappings:
legacy_id
197    5
128    5
192    4
121    4
195    4
139    3
157    3
162    3
199    3
159    3
126    3
112    3
158    3
116    3
163    3
178    3
146    3
131    3
174    3
138    3
125    2
198    2
105    2
166    2
155    2
187    2
179    2
132    2
176    2
190    2
106    2
145    2
111    2
101    2
177    2
129    2
181    2
130    2
109    2
167    2
136    2
175    2
144    2
152    2
141    2
196    2
191    2
183    2
180    2
Name: count, dtype: int64

Handling conflicts by keeping the first mapping encountered for each legacy ID.
Final merged DataFrame after handling one-to-many mappings:
   customer_id_legacy  transaction_id  purchase_amount  customer_rating  \
0                 100            1130       384.920989         4.218168   
1                 100            1113       175.928161         4.000000   
2                 100            1147       384.344822         4.600000   

#Checking for Unmapped IDs

In [15]:
# Check for unmapped legacy IDs
unmapped_legacy_ids = transactions_mapped['customer_id_new'].isnull().sum()
print(f"\nNumber of transactions with unmapped legacy IDs: {unmapped_legacy_ids}")

if unmapped_legacy_ids > 0:
    print("Consider how to handle these unmapped transactions (e.g., dropping them or investigating the missing mappings).")
    # You could filter out these transactions if needed:
    # final_merged_df = final_merged_df.dropna(subset=['customer_id_new'])
    # print("\nFinal merged DataFrame after dropping unmapped transactions:")
    # print(final_merged_df.head())
    # print(final_merged_df.info())


Number of transactions with unmapped legacy IDs: 52
Consider how to handle these unmapped transactions (e.g., dropping them or investigating the missing mappings).


#Handling Missing Values

In [16]:
print("\nMissing values in the final merged DataFrame:")
print(final_merged_df.isnull().sum())


Missing values in the final merged DataFrame:
customer_id_legacy           0
transaction_id               0
purchase_amount              0
customer_rating              0
product_category_encoded     0
purchase_date                0
product_category             0
purchase_month               0
avg_purchase_amount          0
days_since_last_purchase     0
legacy_id                   52
customer_id_new             52
social_media_platform       96
engagement_score            96
purchase_interest_score     96
review_sentiment            96
dtype: int64


#Identify Available Columns

In [22]:
print("\n--- Columns in final_merged_df_cleaned before Feature Engineering ---")
print(final_merged_df_cleaned.columns)


--- Columns in final_merged_df_cleaned before Feature Engineering ---
Index(['customer_id_legacy', 'transaction_id', 'purchase_amount',
       'customer_rating', 'product_category_encoded', 'purchase_date',
       'product_category', 'purchase_month', 'avg_purchase_amount',
       'days_since_last_purchase', 'legacy_id', 'customer_id_new',
       'social_media_platform', 'engagement_score', 'purchase_interest_score',
       'review_sentiment', 'transaction_count', 'normalized_transactions'],
      dtype='object')


#Feature Engineering

# Customer Engagement Score

In [27]:
# Calculate transaction count per customer (using customer_id_new as the identifier)
final_merged_df_cleaned['transaction_count'] = final_merged_df_cleaned.groupby('customer_id_new')['transaction_id'].transform('count')

# Normalize transaction count
final_merged_df_cleaned['normalized_transactions'] = (final_merged_df_cleaned['transaction_count'] - final_merged_df_cleaned['transaction_count'].min()) / (final_merged_df_cleaned['transaction_count'].max() - final_merged_df_cleaned['transaction_count'].min())

# Check for the actual social media activity column
social_activity_column = 'engagement_score'  # Using the column name from your output

if social_activity_column in final_merged_df_cleaned.columns:
    # Normalize social media activity
    final_merged_df_cleaned['normalized_social_activity'] = (final_merged_df_cleaned[social_activity_column] - final_merged_df_cleaned[social_activity_column].min()) / (final_merged_df_cleaned[social_activity_column].max() - final_merged_df_cleaned[social_activity_column].min())

    # Combine them to create an engagement score (adjust weights as needed)
    final_merged_df_cleaned['engagement_score'] = 0.5 * final_merged_df_cleaned['normalized_transactions'] + 0.5 * final_merged_df_cleaned['normalized_social_activity']
    print(f"\nCustomer Engagement Score created (including {social_activity_column}).")
else:
    print(f"\nColumn '{social_activity_column}' not found. Engagement score based on transaction count only.")
    final_merged_df_cleaned['engagement_score'] = final_merged_df_cleaned['normalized_transactions']

print(final_merged_df_cleaned[['customer_id_new', 'transaction_count', 'engagement_score']].head())


Customer Engagement Score created (including engagement_score).
  customer_id_new  transaction_count  engagement_score
3            A116                 50               1.0
4            A116                 50               1.0
5            A116                 50               1.0
6            A116                 50               1.0
7            A116                 50               1.0


#Predictive Behavioral Features

In [28]:
# Ensure 'purchase_date' is in datetime format
if 'purchase_date' in final_merged_df_cleaned.columns:
    final_merged_df_cleaned['purchase_date'] = pd.to_datetime(final_merged_df_cleaned['purchase_date'])
    final_merged_df_cleaned = final_merged_df_cleaned.sort_values(by=['customer_id_new', 'purchase_date'])

    # Moving average of purchase amount (e.g., over the last 3 transactions)
    if 'purchase_amount' in final_merged_df_cleaned.columns:
        final_merged_df_cleaned['moving_average_spent_3'] = final_merged_df_cleaned.groupby('customer_id_new')['purchase_amount'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
        print("\nMoving average of purchase amount created.")
    else:
        print("\nColumn 'purchase_amount' not found, skipping moving average.")

    # Time-based aggregation: Monthly total spending
    final_merged_df_cleaned['month_year'] = final_merged_df_cleaned['purchase_date'].dt.to_period('M')
    monthly_spending = final_merged_df_cleaned.groupby(['customer_id_new', 'month_year'])['purchase_amount'].sum().reset_index()
    monthly_spending = monthly_spending.rename(columns={'purchase_amount': 'monthly_total_spent'})
    final_merged_df_cleaned = pd.merge(final_merged_df_cleaned, monthly_spending, on=['customer_id_new', 'month_year'], how='left')
    print("Monthly total spending aggregated.")

    print(final_merged_df_cleaned[['customer_id_new', 'purchase_date', 'moving_average_spent_3', 'monthly_total_spent']].head())
else:
    print("\nColumn 'purchase_date' not found. Skipping moving averages and time-based aggregation.")
    print(final_merged_df_cleaned[['customer_id_new']].head())


Moving average of purchase amount created.
Monthly total spending aggregated.
  customer_id_new purchase_date  moving_average_spent_3  monthly_total_spent
0            A102    2024-01-21              234.832181           234.832181
1            A102    2024-02-16              314.251987           393.671793
2            A102    2024-05-04              282.677320           454.104469
3            A102    2024-05-05              282.592087           454.104469
4            A107    2024-02-02              221.767060           665.301179


#TF-IDF on Text Data

In [29]:
# Check for text-based columns for TF-IDF
review_col = 'review_sentiment'        # Using the sentiment score column

if review_col in final_merged_df_cleaned.columns:
    print(f"\nColumn '{review_col}' found. TF-IDF is typically used on raw text. This column contains sentiment scores, which is already a numerical feature.")
    # We can directly use the review_sentiment as a feature.
    print("Review sentiment scores will be used as a feature.")
else:
    print("\nNo relevant text columns found for TF-IDF. Skipping TF-IDF.")

print(final_merged_df_cleaned[['customer_id_new', 'review_sentiment']].head())
print(final_merged_df_cleaned.info())


Column 'review_sentiment' found. TF-IDF is typically used on raw text. This column contains sentiment scores, which is already a numerical feature.
Review sentiment scores will be used as a feature.
  customer_id_new review_sentiment
0            A102          Neutral
1            A102          Neutral
2            A102          Neutral
3            A102          Neutral
4            A107          Neutral
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   customer_id_legacy          376 non-null    int64         
 1   transaction_id              376 non-null    int64         
 2   purchase_amount             376 non-null    float64       
 3   customer_rating             376 non-null    float64       
 4   product_category_encoded    376 non-null    int64         
 5   purchase_date               376 

#Export the Final Preprocessed Data

In [32]:
group_number = "7"
output_filename = f'final_customer_data_{7}.csv'
final_merged_df.to_csv(output_filename, index=False)
print(f"\nFinal preprocessed data saved to: {output_filename}")


Final preprocessed data saved to: final_customer_data_7.csv
